### We are assuming that the state vector are in the following order:
$$ x = [E_{qp}\quad E_{dp}\quad \delta\quad \omega\quad V_F\quad V_A\quad V_E]^T $$

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from rushisland1 import *

In [3]:
who

Aex1	 Aex2	 Bex1	 Bex2	 H	 KA1	 KA2	 KC1	 KC2	 
KD1	 KD2	 KE1	 KE2	 KF1	 KF2	 S2_mat	 S3_mat	 T2_mat	 
T3_mat	 TA1	 TA2	 TE1	 TE2	 TF1	 TF2	 Tdp	 Tqp	 
Xd	 Xdp	 Xl	 Xq	 Xqp	 f_Edp1	 f_Edp2	 f_Efd	 f_Eqp1	 
f_Eqp2	 f_Id1	 f_Id2	 f_Iq1	 f_Iq2	 f_VA1	 f_VA2	 f_VE1	 f_VE2	 
f_VF1	 f_VF2	 f_Vdq	 f_delta1	 f_delta2	 f_w1	 f_w2	 nd	 np	 
ratio	 sys_fun	 


In [4]:
import numpy as np
import numdifftools as nd
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import math
%matplotlib inline 
pd.set_option('display.float_format', lambda x: '%.6f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from scipy import optimize
np.set_printoptions(precision=25)

In [5]:
x = np.array([9.8626863024948619e-01,6.1165116423884036e-01,1.2558184342430763e+00,\
              -6.2200648344413002e-30,4.8830798594520859e+00,5.4256442882800945e-03,\
              2.3355826940514226e+00,9.0728914911947700e-01,6.3456098937805228e-01,\
              1.3019464471464004e+00,-3.7087233939572348e-30,2.4973842398326895e+00,\
              4.7569223615860759e-02,2.0985781450456096e+00])
# x = np.append(x,[5.769953284494019,5.400027691631498,1.045418310397626,1.077334874177618])

sol = optimize.root(sys_fun, x, method='hybr')
print(sol.fun)
print(sol.message)
print(sol.x)
print(sol.success)
x = sol.x

[-8.3318801097572719e-17 -1.8722142067877854e-15 -1.4143740482106815e-29
  5.2754717254699771e-17  0.0000000000000000e+00  6.6883227351550231e-15
  0.0000000000000000e+00  0.0000000000000000e+00  0.0000000000000000e+00
 -9.0154746032908310e-31  1.0550943450939954e-17  0.0000000000000000e+00
  0.0000000000000000e+00  0.0000000000000000e+00]
The solution converged.
[ 9.8626863024947953e-01  6.1165116423882404e-01  1.2558184342430694e+00
 -3.7517436858937443e-32  4.8830798594519953e+00  5.4256442882799948e-03
  2.3355826940513795e+00  9.0728914911953285e-01  6.3456098937804051e-01
  1.3019464471463114e+00 -2.3914289123886333e-33  2.4973842398325932e+00
  4.7569223615858913e-02  2.0985781450455301e+00]
True


In [ ]:
# np.abs(f_Vdq(x))

# Notations

$$ \dot{z} = f(z),\quad f(z_0)=0 $$
$$ \dot{\Delta z} = J\Delta z + \frac{1}{2!}  \Delta z^T H \Delta z + H.O.T(z)$$
$$ \dot{\Delta z} = J\Delta z + \frac{1}{2!}  Z_2*B_z^2 + H.O.T(z)$$
Let $w = Q\Delta z$, and ignore the $\Delta$ sign then
$$Q^{-1}\dot{w} = JQ^{-1} w + \frac{1}{2!} (Q^{-1}w)^T H  (Q^{-1}w) + H.O.T(Q^{-1}w)$$
$$\dot{w} = QJQ^{-1} w + Q \frac{1}{2!} w^T (Q^{-T}H Q^{-1})w + Q * H.O.T(Q^{-1}w)$$ 
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + Q \frac{1}{2!} w^T H_Q w + Q * H.O.T(Q^{-1}w)$$
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + Q \frac{1}{2!}  \begin{bmatrix} w^T H_Q[0] w \\  w^T H_Q[1] w \\ ...  \end{bmatrix} + Q * H.O.T(Q^{-1}w)$$ 
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + W_2 B_w^2 + W_3 B_w^3 + ...$$ 
where $W_2$ $W_3$ are constant matrices and $B_w^2$ and $B_w^3$ are column vectors deriving from $w \otimes w$ and $w \otimes w \otimes w$

And we assume that $$w = [u \quad v]^T$$ where $u$ is center variable, $v$ is stable variable

Now assume $B^2_{u}$ is $u\otimes u$ (note that this is a long column vector, so we do not need the $vec$ sign) but ij terms only appear once: e.g. if $u = [u_1 u_2 u_3]^T$, then $B^2_{u}=[u_1^2, u_2u_1, u_3u_1, u_2^2, u_3u_2, u_3^2]$

So we also need two transformation matrices $T$ and $S$ which doing transformations between $B_{uu}$ and $u\otimes u$ like: $$B^2_{u}=T_2 (u\otimes u)$$  
$$u\otimes u=S_2 B^2_{u}$$ 

similarly we will have  
$$B^3_{u}=T_3 (u\otimes B^2_{u})$$  
$$(u\otimes B^2_{u})=S_3 B^3_{u}$$ 

### Now we calculate Jacobian matrix $J$ and  the tranformation matrix $Q$

In [5]:
def Jacobian(f_test, x0):
    """
    This function calculates the 1st order derivative of a function f: R^n -> R
    input: 
        f_test is the function
        x0 where the 1st order want to be calcuated
    return: 1-D row vector
    """
    Jac = np.zeros(x0.shape[0])
    for i in range(x0.shape[0]):
        h = 1e-6
        xp1 = np.array(x0, copy=True) 
        xp1[i] += h
        #print(xp1)
        xp2 = np.array(x0, copy=True) 
        xp2[i] += 2*h
        #print(xp2)
        xm1 = np.array(x0, copy=True) 
        xm1[i] -= h
        #print(xm1)
        xm2 = np.array(x0, copy=True) 
        xm2[i] -= 2*h
        #print(xm2)
        Jac[i] = (-f_test(xp2) + 8*f_test(xp1)- 8*f_test(xm1) + f_test(xm2))/(12*h)
    return Jac

def Hessian(f_test, x0):
    """
    This function calculates the 2nd order derivative of a function f: R^n -> R
    input: 
        f_test is the function
        x0 where the 2nd order want to be calcuated
    return: 2-D matrix
    """
    Hes = np.zeros((x0.shape[0],x0.shape[0]))
    for i in range(x0.shape[0]):
        h = 1e-4
        xp1 = np.array(x0, copy=True) 
        xp1[i] += h
        #print(xp1)
        xp2 = np.array(x0, copy=True) 
        xp2[i] += 2*h
        #print(xp2)
        xm1 = np.array(x0, copy=True) 
        xm1[i] -= h
        #print(xm1)
        xm2 = np.array(x0, copy=True) 
        xm2[i] -= 2*h
        #print(xm2)
        Hes[i] = (-Jacobian(f_test, xp2) + 8*Jacobian(f_test, xp1)- 8*Jacobian(f_test, xm1) + Jacobian(f_test, xm2))/(12*h)
    return Hes



def Trissian(f_test, x0):
    """
    This function calculates the 3rd order derivative of a function f: R^n -> R
    input: 
        f_test is the function
        x0 where the 3rd order want to be calcuated
    return: 3-D matrix
    """
    Trissian = np.zeros((x0.shape[0],x0.shape[0],x0.shape[0]))
    for i in range(x0.shape[0]):
        h = 1e-4
        xp1 = np.array(x0, copy=True) 
        xp1[i] += h
        #print(xp1)
        xp2 = np.array(x0, copy=True) 
        xp2[i] += 2*h
        #print(xp2)
        xm1 = np.array(x0, copy=True) 
        xm1[i] -= h
        #print(xm1)
        xm2 = np.array(x0, copy=True) 
        xm2[i] -= 2*h
        #print(xm2)
        #Trissian[i] = (-nd.Hessian(f_test)(xp2) + 8*nd.Hessian(f_test)(xp1)- 8*nd.Hessian(f_test)(xm1) + nd.Hessian(f_test)(xm2))/(12*h)
        Trissian[i] = (-Hessian(f_test, xp2) + 8*Hessian(f_test, xp1)- 8*Hessian(f_test, xm1) + Hessian(f_test, xm2))/(12*h)
    return Trissian

In [6]:
n_c = 4
n_s = 10
n = x.shape[0]

all_fun = [f_Eqp1,f_Edp1,f_delta1,f_w1,f_VF1,f_VA1,f_VE1,f_Eqp2,f_Edp2,f_delta2,f_w2,f_VF2,f_VA2,f_VE2]
# J= np.array([nd.Jacobian(f)(x).ravel() for f in all_fun])
J = np.array([Jacobian(f,x) for f in all_fun])
#display(pd.DataFrame(J))
lam, v = np.linalg.eig(J.T) # Here we want to calculate the left eigenvecs, so use J_org.T; because use left eigvec makes it easier to calc transformation matrix
lam

array([-1.0139228094316902e+01+15.56811069183762j  ,
       -1.0139228094316902e+01-15.56811069183762j  ,
       -9.8673608539628188e-01 +8.20190087319654j  ,
       -9.8673608539628188e-01 -8.20190087319654j  ,
        1.6491952248287589e-10 +5.536375068433656j ,
        1.6491952248287589e-10 -5.536375068433656j ,
       -7.0192422427770458e+00 +0.j                ,
       -4.9053271488996488e+00 +0.j                ,
       -4.1681806964288803e-10 +0.8705493134731516j,
       -4.1681806964288803e-10 -0.8705493134731516j,
       -1.7968170784180448e+00 +0.8122401596332532j,
       -1.7968170784180448e+00 -0.8122401596332532j,
       -1.7428260326715661e+00 +0.j                ,
       -8.3603203785432467e-01 +0.j                ])

In [7]:
# Compute transformation matrix Q
Q1 = np.block([
    [np.stack((v[:,4].real,v[:,4].imag,v[:,8].real,v[:,8].imag))],  # Here we know the 4-th and 8-th eigenvalues are what we interested
    [np.eye(10),np.zeros((10,4))]
])
J1 = np.dot(np.dot(Q1,J),np.linalg.inv(Q1))
# display(pd.DataFrame(J1))
Q2 = np.block([
    [np.eye(4), np.zeros((4,10))],
    [sp.linalg.solve_sylvester(J1[4:,4:], -J1[0:4,0:4], J1[4:,0:4]), np.eye(10)] #AX+XB=Q
]) 
Q = np.dot(Q2,Q1)
#J_cs = np.dot(np.dot(Q,J_org),np.linalg.inv(Q))
J_cs = np.linalg.multi_dot([Q,J,np.linalg.inv(Q)])
del Q1
del Q2
del J1
display(pd.DataFrame(J_cs))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.000000,-5.536375,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,5.536375,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000
2,0.000000,0.000000,-0.000000,-0.870549,-0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,-0.000000
3,0.000000,-0.000000,0.870549,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000
4,-0.000000,0.000000,-0.000000,-0.000000,-0.728225,-0.011744,-0.435535,0.000000,-0.000000,-0.000000,0.187617,0.177568,0.000421,0.112296
5,-0.000000,-0.000000,-0.000000,-0.000000,0.095547,-5.947079,2.336633,-0.000000,0.000000,-0.000000,-0.000000,-0.003426,1.444611,-1.312854
6,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,376.991118,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,-0.000000
7,0.000000,0.000000,0.000000,0.000000,-0.117252,0.069744,-0.140504,0.000000,-0.000000,0.000000,0.000000,0.027869,-0.044700,0.058241
8,0.000000,-0.000000,0.000000,0.000000,6.085553,0.098145,3.639628,-0.000000,-1.449275,-0.000000,1.473252,-1.483883,-0.003520,-0.938421
9,0.000000,-0.000000,0.000000,0.000000,-7.482345,-6.129778,1.465869,0.000000,0.322061,-22.222222,-0.327389,-2.806158,-3.367794,1.274884


### Now we can 2nd and 3rd order matrices $W_2$ and $W_3$ 
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + W_2 B_w^2 + W_3 B_w^3 + ...$$ 
Since originally we are calculating hessian matrices of $f(z)$, then we need to make transformation from $z$ to $w$, thus
$$ \frac{1}{2} Q Z_2 B_z^2 = \frac{1}{2} Q Z_2[T_2 (z \otimes z)] = \frac{1}{2} Q Z_2 T_2 (Q^{-1} w \otimes Q^{-1} w) = \frac{1}{2} Q Z_2 T_2 (Q^{-1}\otimes Q^{-1})(w\otimes w) = \frac{1}{2} Q Z_2 T_2 (Q^{-1}\otimes Q^{-1})S_2 B_w^2 = W_2 B_w^2$$
Thus
$$\frac{1}{2} Q Z_2 T_2 (Q^{-1}\otimes Q^{-1})S_2 = W_2 $$

For 3rd order
$$\frac{1}{3!} Q Z_3 B_z^3 = \frac{1}{3!} Q Z_3 T_3 (z \otimes B_z^2) = \frac{1}{3!} Q Z_3 T_3 (z \otimes T_2(z \otimes z))$$
where

\begin{align}
&  z \otimes T_2(z \otimes z) \\
& = (I\otimes T_2)(z \otimes z \otimes z) \\
& = (I\otimes T_2)(Q^{-1}w \otimes Q^{-1}w \otimes Q^{-1}w) \\
& = (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(w \otimes w \otimes w) \\
& = (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(w \otimes S_2 B_w^2) \\
& = (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(I \otimes S_2 )(w \otimes B_w^2) \\
& = (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(I \otimes S_2 ) S_3 B_w^3
\end{align}

Thus
$$\frac{1}{3!} Q Z_3 B_z^3 = \frac{1}{3!} Q Z_3 T_3 (I\otimes T_2)(Q^{-1} \otimes Q^{-1} \otimes Q^{-1})(I \otimes S_2 ) S_3 B_w^3 = W_3 B_w^3$$


In [8]:
Z2 = np.zeros((n,n*(n+1)//2))
for i in range(n):
    #hes = nd.Hessian(all_fun[i])(x) # The original Hessian of each f in all_fun
    hes = Hessian(all_fun[i], x)
    hes[np.triu_indices(n,1)] *= 2  # double each element above the main diagonal
    Z2[i] = hes[np.triu_indices(n)] # Keep upper triangular part
Z2 = Z2/2 #divide all elements by 2, which corresponds to *2 in above line

In [9]:
Z3 = np.zeros((n,(n*(n+1)*(n+2)//6)))
Z3_idx = [(i,j,k) for i in range(n) for j in range(i,n) for k in range(j,n)]
for i in range(n):
    t = Trissian(all_fun[i], x)
    Z3[i] = [t[j] for j in Z3_idx]

Z3_Gain = []
for i in Z3_idx:
    val = 1
    for j in range(n_c):
        val *= math.factorial(i.count(j))
    Z3_Gain.append(val)
Z3_Gain = np.diag(1/np.array(Z3_Gain))    
Z3 = Z3.dot(Z3_Gain)
# np.savetxt("Fxxx.csv", Z3, delimiter=",")

# Z3 = np.genfromtxt('Fxxx_goodcase1.csv', delimiter=',',dtype=float)
# display(pd.DataFrame(Z3))

In [10]:
W2 = np.linalg.multi_dot([Q, Z2, T2_mat(n), np.kron(np.linalg.inv(Q),np.linalg.inv(Q)), S2_mat(n)])
display(pd.DataFrame(W2)) # W2 is of dim: n x n(n+1)/2 = 14 x 105


W3 = np.linalg.multi_dot([Q, Z3, T3_mat(n), np.kron(np.eye(n),T2_mat(n)), 
                      np.kron(np.kron(np.linalg.inv(Q),np.linalg.inv(Q)), np.linalg.inv(Q)), 
                      np.kron(np.eye(n),S2_mat(n)), S3_mat(n)])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104
0,-1.740715,-30.355197,-1.081998,-0.840321,-0.135681,-0.383456,0.678616,-0.001506,0.000006,0.000244,0.000752,0.143019,0.539029,-0.013954,-73.617921,5.790683,0.597070,-1.214543,-2.652489,1.766648,0.001071,-0.000004,-0.000174,0.003125,-1.037302,-2.794319,1.845302,-0.577718,-0.521085,-0.046852,-0.136153,0.114848,-0.005556,0.000023,0.000900,-0.000171,0.084056,0.336920,-0.230350,-0.118623,-0.048963,-0.125606,0.092022,0.005619,-0.000023,-0.000910,0.000084,0.042959,0.191903,-0.099430,0.002222,0.000374,0.026024,-0.009509,0.000039,0.001540,0.000028,0.004737,-0.000603,0.004357,0.001687,0.054642,0.006829,-0.000028,-0.001106,-0.000020,-0.000533,0.002604,-0.003313,-0.024038,-0.011403,0.000046,0.001847,0.000034,0.005483,-0.005206,0.009313,0.020725,-0.000169,-0.006714,-0.000123,-0.012797,0.004466,-0.011647,0.000000,0.000027,0.000000,0.000052,-0.000018,0.000047,0.000544,0.000020,0.002073,-0.000723,0.001887,-0.000006,0.000038,-0.000013,0.000034,0.003887,-0.000911,0.023297,0.002183,0.061718,-0.022414
1,-0.019966,-4.245120,1.042360,0.627531,0.290278,-0.129196,0.268420,-0.000607,0.000002,0.000098,-0.001538,-0.290891,0.134176,-0.173301,-56.406978,1.507718,0.139914,0.755452,-0.530425,1.545557,0.000436,-0.000002,-0.000070,-0.006448,0.343361,-0.331760,0.909026,0.523638,0.699534,0.145858,-0.072650,0.016531,-0.002226,0.000009,0.000360,0.000392,-0.152549,0.069233,-0.048887,0.238824,0.116634,-0.063349,0.026977,0.002248,-0.000009,-0.000364,-0.000214,-0.095571,0.037697,-0.028562,0.005214,-0.010454,-0.028954,-0.003807,0.000015,0.000617,0.000011,0.006527,-0.007183,0.016466,0.005038,0.014566,0.002734,-0.000011,-0.000443,-0.000008,-0.004417,0.005296,-0.005968,-0.024185,-0.004565,0.000019,0.000739,0.000014,0.017074,-0.006083,0.018294,0.008297,-0.000067,-0.002688,-0.000049,-0.005123,0.001787,-0.004663,0.000000,0.000011,0.000000,0.000021,-0.000007,0.000019,0.000218,0.000008,0.000830,-0.000289,0.000755,0.000012,0.000015,-0.000005,0.000014,0.002213,-0.003204,-0.021442,0.001332,0.011813,-0.017813
2,-0.423527,-9.962259,3.728154,1.545666,0.166296,0.014184,0.226995,-0.075903,0.000309,0.012290,0.001968,-0.930852,-0.442932,0.030260,-9.756839,-10.086719,-4.996385,0.485802,-1.049249,1.707612,0.054233,-0.000220,-0.008781,0.007137,1.351638,2.465211,-1.388602,1.775904,1.803240,0.092680,0.098360,-0.090794,-0.279174,0.001135,0.045217,0.000390,-0.300806,-0.329485,0.460727,0.496558,-0.020940,0.092695,-0.133912,0.282209,-0.001148,-0.045711,-0.000601,-0.321308,-0.129875,0.083405,0.060539,-0.089679,0.132273,-0.477686,0.001943,0.077371,0.001414,0.149558,-0.020576,0.122336,0.034041,-0.078034,0.343056,-0.001395,-0.055565,-0.001016,-0.146324,0.038737,-0.091171,0.061957,-0.572851,0.002330,0.092786,0.001696,0.222363,-0.049129,0.155446,1.041128,-0.008469,-0.337266,-0.006166,-0.642877,0.224297,-0.585121,0.000017,0.001372,0.000025,0.002615,-0.000912,0.002380,0.027314,0.000999,0.104128,-0.036330,0.094773,-0.000005,0.001904,-0.000664,0.001733,0.097387,-0.069784,0.110629,0.010264,-0.129577,0.101240
3,0.935876,-16.343688,-1.900944,-1.029465,-0.401628,0.338431,-0.720208,0.106323,-0.000436,-0.017286,-0.003012,1.168862,-0.915031,1.037308,149.465106,-15.670164,-7.456009,0.964728,0.798899,-0.138535,-0.075447,0.000310,0.012284,-0.011061,-2.389003,2.109095,-6.323085,-0.930531,-1.096554,-0.216354,0.096088,-0.257148,0.397745,-0.001619,-0.064444,-0.000503,0.563156,-0.528556,0.387243,-0.380934,0.023005,-0.030055,0.009761,-0.403170,0.001639,0.065287,0.000831,0.523579,-0.381016,0.385217,-0.069693,0.093456,-0.169417,0.681636,-0.002772,-0.110406,-0.002018,-0.220144,0.067603,-0.228070,-0.031858,0.090710,-0.489528,0.001991,0.079289,0.001449,0.175321,-0.061293,0.163736,-0.082435,0.817436,-0.003325,-0.132401,

### Now we are able to calculate center manifold up to 2nd order terms, which we denote it by:
$$v = h_{c2}(u) = \color{red}{H_{c2}}B_u^2$$
where $\color{red}{H_{c2}} \in \mathbb{R}^{n_s \times n_c(n_c+1)/2}$ is the matrix we want to find; the subscript $c$ means center manifold (to distinguish it from normal form later), and 2 means 2nd order

Then according to equation:
$$\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + W_2 B_w^2 + W_3 B_w^3 + ...$$ 
if we express the term $W_2 B_w^2$ in form of $u$ and $v$, we will have
$$ \dot{u} = J_c u + f_2(u,v) = J_c u + \color{red}{U_u^2} B_u^2 + \color{red}{U_{uv}} B_{uv} + U_v^2 B_v^2 + ... $$
$$ \dot{v} = J_s v + g_2(u,v) = J_s v + \color{red}{V_u^2} B_u^2 + V_{uv} B_{uv} + V_v^2 B_v^2 + ... $$

Since we can find $\color{red}{H_{c2}}$ by solving the sylvester equation: $\color{red}{J_s H_{c2} + H_{c2} C_2 = -V_u^2}$ so we need to calculate the matrix of $V_u^2$

where $C_2$ comes from
$$ (DB_u^2)J_c u = T_2 D(u \otimes u) J_c u = T_2 \bar{J_c}(u \otimes u) = T_2 \bar{J_c} S_2 B_u^2 = -C_2 B_u^2$$
and
$$\bar{J_c} = (I_n \otimes J_c) + \begin{pmatrix} I_n \otimes J_1\\I_n \otimes J_2 \\ ... \\I_n \otimes J_n \end{pmatrix}$$ where each $J_i$ is the i-th row of matrix $J_c$

Now we construct matrix <font color = 'red'>$V_u^2$, $\bar{J_c}$ $C_2$ $H_{c2}$ </font>

In [11]:
J_c = J_cs[0:n_c,0:n_c]
J_s = J_cs[n_c:,n_c:]
V2_uu = np.zeros((n_s,n_c*(n_c+1)//2))

W2_idx = [(i,j) for i in range(n) for j in range(i,n)]
V2_uu_idx = [(i,j) for i in range(n_c) for j in range(i,n_c)]
V2_uu_idx = [W2_idx.index(i) for i in V2_uu_idx]
V2_uu = np.array([w[V2_uu_idx] for w in W2[n_c:n]])
# display(pd.DataFrame(V2_uu))

J_cbar = np.kron(np.eye(n_c),J_c) + np.array([np.kron(np.eye(n_c),row) for row in J_c]).reshape(-1,n_c**2)
#display(pd.DataFrame(J_cbar))

C2 = - np.linalg.multi_dot([T2_mat(n_c),J_cbar,S2_mat(n_c)])
#display(pd.DataFrame(C2))

H_c2 = sp.linalg.solve_sylvester(J_s, C2, -V2_uu)
# display(pd.DataFrame(H_c2))

### Now we can plug $v = {H_{c2}}B_u^2$ into equation of $\dot{u}$, then we can get up to the 3rd order terms
Since we have 
$$ \dot{u} = J_c u + f_2(u,v) + f_3(u,v) + ... $$
by plugging 
$$v = h_{c2}(u) = H_{c2} B_u^2$$
into the above equation, for third order terms of $u$, we only need:
$$ \dot{u} = J_c u + f_2(u,H_{c2} B_u^2) + f_3(u,0) + ... $$
Let
$$f_2(u,v) = U_u^{2} B_u^2 + U_{uv} B_{uv} + U_v^{2} B_v^2$$
and 
$$f_3(u,0) = U_u^{3} B_u^3$$
Then we can see that $B_{uv}$ is in 3rd order of $u$. $B_v^2$ is in 4th order of $u$, which is ignored here.
So the center manifold up to 3rd order of $u$ could be expressed as:
$$\dot{u} = J_c u + U_u^{2} B_u^2 + U_{uv} B_{uv} + U_u^{3} B_u^3 + O(u^4) $$
Since 
$$B_{uv} = u \otimes v = u \otimes H_{c2} B_u^2 = (I_{n_c} \otimes H_{c2})(u \otimes B_u^2) = (I_{n_c} \otimes H_{c2})S_3 B_u^3$$
Thus
$$\dot{u} = J_c u + U_u^{2} B_u^2 + U_{uv} (I_{n_c} \otimes H_{c2})S_3 B_u^3 + U_u^{3} B_u^3 + O(u^4) = J_c u + \color{red}{U_u^{2}} B_u^2 + [\color{red}{U_{uv}} (I_{n_c} \otimes H_{c2})\color{red}{S_3} + \color{red}{U_u^{3}}] B_u^3 + O(u^4)$$

So here in order to get the know the equation of $\dot{u}$ up to 3rd order, we need to know the following matrices: <font color = 'red'>$U_u^{2}$, $U_{uv}$, $S_3$, $U_u^{3}$</font>.

Here in order to know $U_u^{3}$, which comes from $f_3(u,0)$, we need to calculate matrix $W_3$ in equation $\dot{w} = \begin{bmatrix} J_c & 0\\ 0 & J_s \end{bmatrix} w + W_2 B_w^2 + W_3 B_w^3 + ...$

### Firstly we calculate $U_u^2$ and $U_{uv}$. Both of them comes from $W_2$

In [12]:
Uu2_idx = [(i,j) for i in range(n_c) for j in range(i,n_c)]
Uu2_idx = [W2_idx.index(i) for i in Uu2_idx]
Uu2 = np.array([w[Uu2_idx] for w in W2[0:n_c]])
#display(pd.DataFrame(Uu2))
Uuv_idx = [(i,j) for i in range(n_c) for j in range(n_c,n)]
Uuv_idx = [W2_idx.index(i) for i in Uuv_idx]
Uuv = np.array([w[Uuv_idx] for w in W2[0:n_c]])
#display(pd.DataFrame(Uuv))

### Now we are ready to calculate matrix $U_u^3$. 
But in order to calculate it, we need to calculate the 3rd order derivative matrix $W_3$, then formulize $U_u^3$

Now we have known the following matrices: <font color = 'red'>$U_u^{2}$, $U_{uv}$, $S_3$  and $U_u^{3}$ </font>

Then we can calculate 
$$U_{uc}^3 = U_{uv} (I_{n_c} \otimes H_{c2})S_3 + U_u^{3}$$ 
in the center manifold:
$$\dot{u} =  J_c u + U_u^{2} B_u^2 + [U_{uv} (I_{n_c} \otimes H_{c2})S_3 + U_u^{3}] B_u^3 + O(u^4) = J_c u + U_u^{2} B_u^2 + U_{uc}^3 B_u^3 + O(u^4)$$

In [13]:
W3_idx = [(i,j,k) for i in range(n) for j in range(i,n) for k in range(j,n)]
Uu3_idx = [(i,j,k) for i in range(n_c) for j in range(i,n_c) for k in range(j,n_c)]
Uu3_idx = [W3_idx.index(i) for i in Uu3_idx]
Uu3 = np.array([w[Uu3_idx] for w in W3[0:n_c]])
#display(pd.DataFrame(Uu3))
Uuc3 = np.linalg.multi_dot([Uuv, np.kron(np.eye(n_c), H_c2), S3_mat(n_c)]) + Uu3

### <font color='red'>Now we've got the flow $\dot{u}$ on the center manifold up to 3rd order, the next step is to calculate the normal form</font>

<font color='red'>$$\dot{u} = J_c u + U_u^{2} B_u^2 + U_{uc}^3 B_u^3 + O(u^4)$$
至此，我们只需要研究这个方程，我们已经知道$J_c$, $U_u^{2}$, $U_{uc}$,那么接下来就是要算normalform
</font>
#### First, we calculate the normal form up to the 2nd order:
$$u = y + \color{red}{H_{2n}} B_y^2$$
where we want to find matrix $\color{red}{H_{2n}}\in \mathbb{R}^{n_c \times n_c(n_c+1)/2}$

Then by plugging it into the above equation, we will get:
$$(I + H_{2n} DB_y^2) \dot{y} = J_c (y + H_{2n} B_y^2) + U_u^2 B_u^2 + U_{uc}^3 B_u^3 + O(u^4)$$
<font color='red'>$$\dot{y} = (I + H_{2n} DB_y^2)^{-1} [J_c y + J_c H_{2n} B_y^2 + U_u^2 B_u^2 + U_{uc}^3 B_u^3 + O(u^4)]$$</font>
Since by some derivation, we know:
$$B_u^2 = B_y^2 + T_2\bar{H}(I\otimes S_2)S_3 B_y^3$$
where
$$\bar{H} = (I_n \otimes H) + \begin{pmatrix} I_n\otimes h_1 \\I_n\otimes h_2 \\ \vdots \\ I_n\otimes h_n \\ \end{pmatrix}$$
$H = H_{2n}T_2$ ($T_2=T_2(n_c)$) and $h_1$ ... $h_n$ are rows of matrix $H$.
In the above equations, if we assume $u\in \mathbb{R}^{n_c}$, then $H_{2n} \in \mathbb{R}^{n_c \times \frac{n_c(n_c+1)}{2}}$, $H \in \mathbb{R}^{n_c\times n_c^2}$, $\bar{H} \in \mathbb{R}^{n_c^2 \times n_c^3}$, $S_2=S_2(n_c)$, $S_3=S_3(n_c)$

Then
<font color='red'>$$\dot{y} = (I - H_{2n} DB_y^2)  [J_c y + J_c H_{2n} B_y^2 + U_u^2 B_y^2 + U_u^2 T_2\bar{H}(I\otimes S_2)S_3 B_y^3 + U_{uc}^3 B_u^3 + O(u^4)]$$</font>
**Then up to 2nd order terms on the right hand side is**:
$$\dot{y} = J_c y + J_c H_{2n} B_y^2 + U_u^2 B_y^2 - H_{2n} DB_y^2 J_c y + O(y^3)$$
If we denote $- DB_y^2 J_c y = \color{red}{C_2} B_y^2$ as we did in calculation of center manifold, then:
$$\dot{y} = J_c y + [J_c H_{2n} + U_u^2 + H_{2n} C_2] B_y^2 + O(y^3)$$
Due to resonance terms, $J_c H_{2n} + H_{2n} C_2 + U_u^2 $ might not be totally eliminated, so we let
<font color='red'>$$J_c H_{2n} + H_{2n} C_2 = - U_u^2 + R_{2n} $$</font>
where matrix $R_{2n}$ represents resonance terms in normal form.

Now the problem is trying to solve for $H_{2n}$ which makes $R_{2n}$ as simple as possible, where $J_c$, $U_{u}^2$ and $C_2$ are known

Since $J_c \in \mathbb{R}^{n_c\times n_c}$ , $H_{2n} \in \mathbb{R}^{n_c\times \frac{n_c(n_c+1)}{2}}$  and $C_2 \in \mathbb{R}^{\frac{n_c(n_c+1)}{2} \times \frac{n_c(n_c+1)}{2}}$
First we could transform the equation into:
$$\underbrace{(I_{\frac{n_c(n_c+1)}{2}}\otimes J_c + C_2^T \otimes I_{n_c})}_\text{$\Lambda_2$} \cdot \text{vec} (H_{2n}) = -\text{vec}(U_u^2) + \text{vec}(R_{2n})$$


Since we know that for Hopf Bifurcation, the 2nd order terms in normal form will all be eliminated, so $\Lambda$ should be a full rank matrix and the resonance term $R_{2n}$ will not exist, which is also verified below. Then we could solve for $H_{2n}$ by directly solving the Sylvester equation $J_c H_{2n} + H_{2n} C_2 = - U_u^2$

In [14]:
Lambda2 = np.kron(np.eye(n_c*(n_c+1)//2),J_c) + np.kron(C2.T, np.eye(n_c))
# Since we know 2nd order will be eliminated, so Lambda should be full rank
assert(np.linalg.matrix_rank(Lambda2) == Lambda2.shape[0])
# So R_2n will be zero and H_2n could be solved directly from the Sylvester Equation
H_2n = sp.linalg.solve_sylvester(J_c, C2, -Uu2)
#display(pd.DataFrame(H_2n))
R_2n = np.dot(J_c,H_2n) + np.dot(H_2n,C2) + Uu2

In [15]:
# dummy1, dummy2 = np.linalg.eig(Lambda2)
# print(dummy1)
# display(pd.DataFrame(R_2n))
R_2n

array([[-6.8167693711984612e-14, -3.5527136788005009e-15,
        -4.4408920985006262e-16, -2.1094237467877974e-15,
         1.4210854715202004e-14,  4.4408920985006262e-15,
        -1.3322676295501878e-15,  3.3306690738754696e-16,
         4.4408920985006262e-16, -4.1633363423443370e-17],
       [-7.2681444196476264e-14, -3.1086244689504383e-14,
        -5.1070259132757201e-15, -3.9968028886505635e-15,
         4.9737991503207013e-14,  2.6645352591003757e-15,
        -1.0824674490095276e-15, -2.2204460492503131e-16,
        -7.7715611723760958e-16, -1.1102230246251565e-16],
       [-6.1062266354383610e-16, -5.1514348342607263e-14,
         2.2204460492503131e-15,  4.4408920985006262e-16,
         2.4868995751603507e-14, -2.3092638912203256e-14,
         2.6645352591003757e-15,  1.1102230246251565e-15,
         4.4408920985006262e-16, -1.0547118733938987e-15],
       [ 7.1609385088322597e-14, -3.1974423109204508e-14,
         1.9984014443252818e-15,  2.2204460492503131e-15,
        -2.

### Next, we calculate the normal form up to the 3rd order terms
From previous equation:
$$\dot{y} = (I + H_{2n} DB_y^2)^{-1} [J_c y + J_c H_{2n} B_y^2 + U_u^2 B_u^2 + U_{uc}^3 B_u^3 + O(u^4)]$$
<font color='red'>$$\dot{y} = (I - H_{2n} DB_y^2)  [J_c y + J_c H_{2n} B_y^2 + U_u^2 B_y^2 + U_u^2 T_2\bar{H}(I\otimes S_2)S_3 B_y^3 + U_{uc}^3 B_u^3 + O(u^4)]$$</font>
Since if we only consider 3rd order terms, $U_{uc}^3 B_u^3$ can be expressed as:
$$ U_{uc}^3 B_u^3 = U_{uc}^3 B_y^3$$
So 
$$\dot{y} = [I - \underbrace{H_{2n} DB_y^2}_\text{O(y)} + \underbrace{(H_{2n} DB_y^2)(H_{2n} DB_y^2)}_\text{O(y^2)}+O(y^3)] [J_c y + \underbrace{(J_c H_{2n} + U_u^2) B_y^2}_\text{O(y^2)} + \underbrace{[U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3}_\text{O(y^3)} + O(u^4)]$$
Thus the 3rd order terms on the right hand side is:

\begin{align}
& [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3 - H_{2n} DB_y^2 (J_c H_{2n} + U_u^2) B_y^2 + (H_{2n} DB_y^2)(H_{2n} DB_y^2)J_c y \\
= & [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3 - H_{2n} DB_y^2 [(J_c H_{2n} + U_u^2) B_y^2 - H_{2n} DB_y^2 J_c y] \\
= & [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3 - H_{2n} DB_y^2 [(J_c H_{2n} + U_u^2) B_y^2 + H_{2n} C_2 B_y^2] \\
= & [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3] B_y^3 - H_{2n} \underbrace{DB_y^2 R_{2n} B_y^2}_\text{O(y^3)} 
\end{align}

where the sizes of matrices are:
\begin{align}
& H_{2n}, U_{u2} --  n_c \times \frac{n_c(n_c+1)}{2}  \\
& T_2(n_c) -- \frac{n_c(n_c+1)}{2} \times n_c^2  \\
& \bar{H}, \bar{R} -- n_c^2 \times n_c^3  \\
& I\otimes S_2(n_c) -- (n_c \times n_c) \otimes (n_c^2 \times \frac{n_c(n_c+1)}{2})  \\
& S_3(n_c) -- \frac{n_c^2 n_c(n_c+1)}{2} \times \frac{n_c(n_c+1)(n_c+2)}{3!}  \\
& U_{uc}^3 -- n_c \times \frac{n_c(n_c+1)(n_c+2)}{3!}  \\
\end{align}

So we only need to calculate 
$$ DB_y^2 R_{2n} B_y^2 = T_2 \bar{R} (I \otimes S_2) S_3 B_y^3$$
where $$\bar{R} = (I_n\otimes R) + \begin{pmatrix} I_n\otimes r_1 \\ I_n\otimes r_2 \\ \vdots \\ I_n\otimes r_n \end{pmatrix} $$
where $R = R_{2n}T_2$ and $r_i$ are rows of $R$

Thus the 3rd order terms are:
$$N_y^3 B_y^3 = [U_u^2 T_2\bar{H}(I\otimes S_2)S_3 + U_{uc}^3 - H_{2n} T_2 \bar{R} (I \otimes S_2) S_3 ]B_y^3$$
where
$$\bar{H} = (I_n \otimes H) + \begin{pmatrix} I_n\otimes h_1 \\I_n\otimes h_2 \\ \vdots \\ I_n\otimes h_n \\ \end{pmatrix}$$
$H = H_{2n}T_2$ and $h_1$ ... $h_n$ are rows of matrix $H$

Now the normal form is:
$$\dot{y} = J_c y + R_{2n}B_y^2 + N_y^3 B_y^3 + O(y^4)$$

Now we want to simply $N_y^3$ as many as possible, so we consider another 3rd order nonlinear transformation 
$$ y = z + H_{3n}B_z^3$$

Here if we assume $y,z \in \mathbb{R}^{n_c}$ then $H_{3n} \in \mathbb{R}^{n_c \times \frac{n_c(n_c+1)(n_c+2)}{3!}}$

Similarly, we will have 
$$\dot{z} = J_c z + R_{2n}B_z^2 + R_{3n} B_z^3 + O(z^4)$$
where
$$R_{3n} = J_c H_{3n} + H_{3n}C_3 + N_y^3$$
where 
$$C_3 B_z^3 = -DB_z^3 J_c z$$ and $$C_3 = -T_3(I_n\otimes T_2)\bar{J}(I_n\otimes S_2)S_3$$
where
$$\bar{J} = (I_{n^2} \otimes J_c) + I_n \otimes \begin{pmatrix} I_n\otimes jc_1 \\I_n\otimes jc_2 \\ \vdots \\ I_n\otimes jc_n \\ \end{pmatrix} + \begin{pmatrix} I_{n^2}\otimes jc_1 \\I_{n^2}\otimes jc_2 \\ \vdots \\ I_{n^2}\otimes jc_n \\ \end{pmatrix}$$

$jc_i$ are rows of matrix $J_c$

So here we have $R_{3n} \in \mathbb{R}^{n_c \times \frac{n_c(n_c+1)(n_c+2)}{3!}}$, $C_3 \in \mathbb{R}^{\frac{n_c(n_c+1)(n_c+2)}{3!} \times \frac{n_c(n_c+1)(n_c+2)}{3!}}$, 

So in order to calculate $R_{3n}$, first we need to calculate $N_y^3$.
Let's calcuate $\bar{H}$ and $\bar{R}$ first

In [16]:
H_bar = np.kron(np.eye(n_c),np.dot(H_2n,T2_mat(n_c))) + np.array([np.kron(np.eye(n_c),row) for row in np.dot(H_2n,T2_mat(n_c))]).reshape(-1,n_c**3)
#display(pd.DataFrame(H_bar))
R_bar = np.kron(np.eye(n_c),np.dot(R_2n,T2_mat(n_c))) + np.array([np.kron(np.eye(n_c),row) for row in np.dot(R_2n,T2_mat(n_c))]).reshape(-1,n_c**3)
#display(pd.DataFrame(R_bar))
Ny3 = np.linalg.multi_dot([Uu2,T2_mat(n_c),H_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)]) + Uuc3 - np.linalg.multi_dot([H_2n,T2_mat(n_c),R_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)])
#display(pd.DataFrame(Ny3))
J_bar = np.kron(np.eye(n_c**2),J_c) + np.kron(np.eye(n_c), np.array([np.kron(np.eye(n_c),row) for row in J_c]).reshape(-1,n_c**2)) + np.array([np.kron(np.eye(n_c**2),row) for row in J_c]).reshape(-1,n_c**3) 
#display(pd.DataFrame(J_bar))
C3 = -np.linalg.multi_dot([T3_mat(n_c),np.kron(np.eye(n_c),T2_mat(n_c)),J_bar,np.kron(np.eye(n_c),S2_mat(n_c)),S3_mat(n_c)])
#display(pd.DataFrame(C3))

Then we tranform the equation:
$$R_{3n} = J_c H_{3n} + H_{3n}C_3 + N_y^3$$
into:
$$(I_{\frac{n_c(n_c+1)(n_c+2)}{3!}} \otimes J_c + C_3^T \otimes I_{n_c}) * \text{vec}(H_{3n}) = \text{vec}(R_{3n}) - \text{vec}(N_y^3)$$
Let
$$\Lambda_3 = I_{\frac{n_c(n_c+1)(n_c+2)}{3!}} \otimes J_c + C_3^T \otimes I_{n_c}$$
<font color='red'>
Here we are assuming the $vec$ operation is taking the column vectors of the matrix. However, it is more easily for us to interprete the results if we assume that $vec$ is taking the row vectors of the matrix. But here the $\Lambda_3$ would be changing to
    $$ \Lambda_3 = J_c \otimes I_{\frac{n_c(n_c+1)(n_c+2)}{3!}} + I_{n_c} \otimes C_3^T$$
</font>

In [17]:
Lambda3 = np.kron(J_c,np.eye(n_c*(n_c+1)*(n_c+2)//6)) + np.kron(np.eye(n_c),C3.T)
np.linalg.matrix_rank(Lambda3)

80

In [18]:
lam, _ = np.linalg.eig(Lambda3)
lam

array([-3.2983982212186902e-10+2.2145500273734644e+01j,
       -3.2983982212186902e-10-2.2145500273734644e+01j,
       -3.2984226461252319e-10+1.1072750136867327e+01j,
       -3.2984226461252319e-10-1.1072750136867327e+01j,
       -9.1157847936784719e-10+1.7479674518774171e+01j,
       -9.1157847936784719e-10-1.7479674518774171e+01j,
        2.5189406116510327e-10+1.7479674518774150e+01j,
        2.5189406116510327e-10-1.7479674518774150e+01j,
       -9.1157389969787062e-10+1.5738575891827830e+01j,
       -9.1157389969787062e-10-1.5738575891827830e+01j,
        2.5189406116510327e-10+1.5738575891827880e+01j,
        2.5189406116510327e-10-1.5738575891827880e+01j,
       -3.2983941967195182e-10+1.1072750136867304e+01j,
       -3.2983941967195182e-10-1.1072750136867304e+01j,
       -3.2983837883193701e-10+1.2813848763813656e+01j,
       -3.2983837883193701e-10-1.2813848763813656e+01j,
        8.3362960978661249e-10+1.2813848763813638e+01j,
        8.3362960978661249e-10-1.281384876381363

In [ ]:
# Lambda3.shape

In [ ]:
# pd.DataFrame(Lambda3)

Now Since we know that the column vector $-vec(N_y^3)$ is in the column space of matrix $\Lambda_3$ but vector $vec(R_3^n)$ is not. This means vector $vec(R_3^n)$ is in the complement space of the column space of matrix $\Lambda_3$. Since we have already know what the normal form with 3rd order resonance terms looks like (shown below), so we could easiliy know the basis of the space where vector $vec(R_3^n)$ is in.

$$ \dot{z}=i\omega_1z+a_1z^2\bar{z}+a_2zw\bar{w}+\ldots $$ 
$$ \dot{w}=i\omega_2w+b_1w^2\bar{w}+b_2z\bar{z}w+\ldots $$


$$ \dot{x_1}=-\omega_1x_2+(a_1^Rx_1-a_1^Ix_2)(x_1^2+x_2^2)+(a_2^Rx_1-a_2^Ix_2)(x_3^2+x_4^2)$$
$$ \dot{x_2}=\omega_1x_1+(a_1^Rx_2+a_1^Ix_1)(x_1^2+x_2^2)+(a_2^Rx_2+a_2^Ix_1)(x_3^2+x_4^2) $$
$$ \dot{x_3}=-\omega_2x_4+(b_1^Rx_3-b_1^Ix_4)(x_3^2+x_4^2)+(b_2^Rx_3-b_2^Ix_4)(x_1^2+x_2^2)$$ 
$$ \dot{x_4}=\omega_2x_3+(b_1^Rx_4+b_1^Ix_3)(x_3^2+x_4^2)(b_2^Rx_4+b_2^Ix_3)(x_1^2+x_2^2)$$

In [19]:
L3 = np.zeros((80,8))
L3[[0,4,21,30],[0,0,0,0]] = 1 # a1_R
L3[[7,9,33,35],[2,2,2,2]] = 1 # a2_R
L3[[56,58,77,79],[4,4,4,4]] = 1 # b1_R
L3[[42,51,63,72],[6,6,6,6]] = 1 # b2_R

L3[[1,10],[1,1]] = -1
L3[[20,24],[1,1]] = 1   #a1_I
L3[[13,15],[3,3]] = -1
L3[[27,29],[3,3]] = 1    #a2_I
L3[[57,59],[5,5]] = -1
L3[[76,78],[5,5]] = 1    #b1_I
L3[[43,52],[7,7]] = -1
L3[[62,71],[7,7]] = 1    #b2_I

# np.linalg.matrix_rank(L3)
# print(np.linalg.matrix_rank((np.concatenate((Lambda3,L3),axis=1))))
# assert(np.linalg.matrix_rank((np.concatenate((Lambda3,L3),axis=1))) == Lambda3.shape[0])

In [ ]:
# n_c = 4
# for i,v in enumerate([(i,j,k) for i in range(1,n_c+1) for j in range(i,n_c+1) for k in range(j,n_c+1)]):
#     print(i,v)

In [20]:
P, L, U = sp.linalg.lu(Lambda3,permute_l=False)
L = P.dot(L)
L_inv = np.linalg.inv(L)

U_zero_rows = np.where(abs(np.diag(U)) < 1e-8)[0]
L2 = L_inv[U_zero_rows,:]

In [21]:
U_zero_rows

array([24, 29, 30, 35, 71, 72, 78, 79], dtype=int64)

In [22]:
# pd.DataFrame(U[U_zero_rows,:].T)
np.where(abs(U[U_zero_rows,:]) > 1e-12)

(array([0, 1, 1, 2, 3, 4, 5, 6, 7], dtype=int64),
 array([24, 29, 34, 30, 35, 71, 72, 78, 79], dtype=int64))

In [23]:
theta =np.linalg.inv(L2.dot(L3)).dot(L2).dot(Ny3.reshape((1,-1)).T)
theta

array([[-7.6453202940235929e+02],
       [-2.7819949230178008e+03],
       [ 3.0371533643859081e+00],
       [-4.6722203607851313e+01],
       [-2.3480988761991792e+00],
       [ 1.9421201134689552e+00],
       [-2.0311805290540775e+01],
       [ 7.7641609634646966e+02]])

In [ ]:
# array([[-4.5930653532974820e+03],
#        [-2.3584980960201228e+04],
#        [-2.1130687624186837e+01],
#        [-5.3841290001287825e+02],
#        [ 1.1753983489160613e+02],
#        [ 2.2105721704826431e+02],
#        [-2.1324594984381474e+02],
#        [ 1.0068823367999465e+04]])

In [ ]:
# array([[ -402.60162741028944 ],
#        [-2782.72622762334    ],
#        [   10.183848369229011],
#        [  -43.13591682277431 ],
#        [   -4.921788501741   ],
#        [   -2.897816138563459],
#        [  -31.030564497181842],
#        [  824.1382749183861  ]])

In [ ]:
# array([[-6.3470132634001288e+02],
#        [-2.6147414977491276e+03],
#        [ 5.3776054423841293e+00],
#        [-4.4578495401119525e+01],
#        [-2.6242139731366096e+00],
#        [ 2.3398918316453976e-01],
#        [-7.1408486518023437e+00],
#        [ 7.7413993164544945e+02]])

In [ ]:
# fig,ax = plt.subplots(figsize=(10,10))
# ax.spy(U, precision=0.1, markersize=5)
# plt.grid(True)

$$ \dot{r_1}=\mu_1 r_1+a_1^R r_1^3+a_2^R r_1 r_2^2+\ldots $$ 
$$ \dot{r_2}=\mu_2 r_2+b_1^R r_2^3+b_2^R r_1^2 r_2+\ldots $$ 

In [ ]:
# a1_R, a2_R, b1_R, b2_R
# ab_R = theta[0::2]
# ab_R